# M-PESA Statement Analysis

## 1. Load and Clean the Dataset

In [1]:
# Load necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber
import csv


In [2]:
# Open the PDF file
pdf_file = "MPESA_Statement_2024-01-01_to_2024-12-24_2547xxxxxx374_unlocked.pdf"

# Extract the table data
with pdfplumber.open(pdf_file) as pdf:
    all_data = []
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            all_data.extend(table)

# Set the appropriate column names for the dataframe
columns = ["Receipt No.", "Completion Time", "Details", "Transaction Status", "Paid In", "Withdrawn", "Balance"]
transactions = pd.DataFrame(all_data[1:], columns=columns)  # Skip header row from the data
print(transactions.head())

                 Receipt No. Completion Time     Details Transaction Status  \
0                SEND MONEY:            0.00  102,383.00               None   
1            RECEIVED MONEY:      175,743.00        0.00               None   
2             AGENT DEPOSIT:       10,515.00        0.00               None   
3          AGENT WITHDRAWAL:            0.00   16,887.00               None   
4  LIPA NA M-PESA (PAYBILL):            0.00   40,738.00               None   

  Paid In Withdrawn Balance  
0    None      None    None  
1    None      None    None  
2    None      None    None  
3    None      None    None  
4    None      None    None  


In [ ]:
# Save to CSV
csv_file = "mpesa_statement.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(all_data)

print(f"Data saved to {csv_file}")

In [4]:
# Load the extracted CSV
df = pd.read_csv("mpesa_statement.csv")
df.head()

'''# Rename columns
df.columns = ["Completion Time", "Details", "Paid In", "Withdrawn", "Balance"]

# Convert financial columns to numeric
for col in ["Paid In", "Withdrawn", "Balance"]:
    df[col] = df[col].str.replace(",", "").astype(float)

# Save the cleaned CSV
cleaned_csv = "cleaned_mpesa_statement.csv"
df.to_csv(cleaned_csv, index=False)
print(f"Cleaned data saved to {cleaned_csv}")'''

ParserError: Error tokenizing data. C error: Expected 3 fields in line 10, saw 7


### Data Cleaning

In [ ]:
# Convert financial columns to numeric values
columns_to_clean = ['Paid In', 'Withdrawn', 'Balance']
for col in columns_to_clean:
    data[col] = data[col].replace('[^0-9.]', '', regex=True).astype(float)

# Convert 'Completion Time' to datetime format
data['Completion Time'] = pd.to_datetime(data['Completion Time'])

# Check for missing values
data.isnull().sum()

## 2. Basic Analysis of Paid In and Withdrawn Amounts

In [ ]:
# Summarize paid in and withdrawn amounts
summary = data[['Paid In', 'Withdrawn']].sum()
summary.plot(kind='pie', autopct='%1.1f%%', colors=['green', 'red'], labels=['Paid In', 'Withdrawn'])
plt.title('Summary of Transactions')
plt.ylabel('')
plt.show()

## 3. Top 10 Sources of Incoming Payments and Withdrawals

In [ ]:
# Top 10 sources of Paid In
top_paid_in_sources = data.groupby('Details')['Paid In'].sum().nlargest(10)
top_paid_in_sources.plot(kind='bar', color='green')
plt.title('Top 10 Sources of Paid In')
plt.ylabel('Amount')
plt.show()

# Top 10 sources of Withdrawn
top_withdrawn_sources = data.groupby('Details')['Withdrawn'].sum().nlargest(10)
top_withdrawn_sources.plot(kind='bar', color='red')
plt.title('Top 10 Sources of Withdrawn')
plt.ylabel('Amount')
plt.show()

## 4. Query Transactions Above a Given Threshold

In [ ]:
# Query for transactions above a given threshold (e.g., 10,000)
threshold = 10000
high_value_paid_in = data[data['Paid In'] > threshold].sort_values(by='Paid In', ascending=False)
high_value_withdrawn = data[data['Withdrawn'] > threshold].sort_values(by='Withdrawn', ascending=False)

# Display results
high_value_paid_in.head(), high_value_withdrawn.head()

## 5. Query Transactions Related to a Specific Customer or Merchant

In [ ]:
# Query for a specific customer or merchant
search_term = 'Customer Name or Merchant'
filtered_data = data[data['Details'].str.contains(search_term, case=False, na=False)]
filtered_data.head()

## 6. Balance Trend Over Time

In [ ]:
# Extract month from 'Completion Time'
data['Month'] = data['Completion Time'].dt.to_period('M')

# Group by month and calculate average balance
balance_trend = data.groupby('Month')['Balance'].mean()
balance_trend.plot(kind='line', marker='o')
plt.title('Balance Trend Over Time')
plt.xlabel('Month')
plt.ylabel('Balance')
plt.grid(True)
plt.show()